### shm - Shared Memory

I created this shared memory interface to pass variables amongst processes in linux for a detector firmware I was writing.  It uses the ``shmget`` and ``shmat`` C interface under the hood, but that's all so non-semantic when you code it that you shouldn't have to worry about it.

#### Example

My fiance is watching the tv show Sherlock Holmes behind me, so this example is themed on that show.

First, we have to create some server for the shared memory.  This server has to be open for the client process to read it, so we'll just keep it open indefinitely.  We also have to define a unique identifier for the shared memory server.

In [3]:
from cppm import cppmagic

In [10]:
%set_compiler g++ --std=c++11 -I/usr/include/glib-2.0 -I/usr/lib/x86_64-linux-gnu/glib-2.0/include -lglib-2.0

In [11]:
%%runcppnb shm_server
#include "../ah_shm.h"
#include "../ah_signal.h"
#include <string>

int run = 1;

void cleanup(int signum){
    run = 0;
}

int main(void){
    // create the shm server
    shm_server<std::string> *sherlock = new shm_server<std::string>(5678);
    // latch a cleanup function onto the SIGTERM signal
    ah_signal(SIGTERM, &cleanup);

    while (run){
        sherlock->set("is a good guy!");
        sleep(1);
        sherlock->set("is a bad guy!");
        sleep(1);
        sherlock->set("is dead!");
        sleep(1);
        sherlock->set("is now alive again!");
        sleep(1);
    }
    
    delete sherlock;
    return 0;
}

Now, to connect to the memory.  We have to attach using the same key, which is a little bit of a paint, but if you use macros effectively, it'll just be like pointing at a file.  This client will be way faster than using files though, for time intensive processes, use these ``shm``s.

In [12]:
%%runcpp shm_client
#include "../ah_shm.h"
#include <string>

int main(void){
    shm_client<std::string> *holmes = new shm_client<std::string>(5678);
    for(int i=1; i<5; i++){
        const char* desc = holmes->get().c_str();
        printf("Episode %d in which Sherlock %s\n", i, desc);
        sleep(1);
    }
    delete holmes;
    return 0;
}

output: 


RuntimeError: Segmentation fault (core dumped)


Now, we just have to stop the server to make sure we're not leaving too many things open.

In [ ]:
%killall

Just a note:  I'm still working on the cleanup of this.  To check that I've removed all of the attached shared memory, you can use the bash line

In [13]:
!ipcs


------ Message Queues --------
key        msqid      owner      perms      used-bytes   messages    

------ Shared Memory Segments --------
key        shmid      owner      perms      bytes      nattch     status      
0x00000000 425984     ahagen     600        524288     2          dest         
0x00000000 1179649    ahagen     600        524288     2          dest         
0x00000000 917506     ahagen     600        524288     2          dest         
0x00000000 42270723   ahagen     600        2097152    2          dest         
0x00000000 753668     ahagen     600        1048576    2          dest         
0x00000000 851973     ahagen     600        524288     2          dest         
0x00000000 1572870    ahagen     600        524288     2          dest         
0x00000000 1474567    ahagen     600        524288     2          dest         
0x00000000 1343496    ahagen     600        524288     2          dest         
0x00000000 42598409   ahagen     600        524288     2   

and to remove it, find the ``KEY`` corresponding to it and use ``ipcrm``.  Note that the KEY is usually the hexadecimal representation of the ``key`` you used in ``c++``, so you can just use that:

In [15]:
!ipcrm -M 5678